# NBA Season Stats Management

In [220]:
import asyncio
import pandas as pd
from pony.orm import *

## Create the database connection

In [221]:
db = Database()
# PostgreSQL
db.bind(provider='postgres', user='postgres', password='05041997', host='localhost', database='nba')

## Map All the entity of logical model through PonyORM

In [222]:
from datetime import date


class Arena(db.Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str,30)
    capacity = Optional(int, nullable=True)
    city = Optional(str,50, nullable=True)
    team = Optional(lambda : Team)

class Season(db.Entity):
    season = PrimaryKey(int)
    start_date = Required(date, unique=False)
    end_date = Required(date, unique=False)
    team_season = Set(lambda : Team_Season)

class Team_Season(db.Entity):
    season = Required(Season)
    team = Required(lambda : Team)
    coach = Optional(lambda : Coach)
    stats = Optional(lambda : TeamStats, cascade_delete=True)
    player = Set(lambda : Player)
    PrimaryKey(season, team)

class Team(db.Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str, 50, unique=True, nullable=False)
    abbr = Optional(str, 5, unique=False, nullable=False)
    nickname = Optional(str, 50, unique=False, nullable=False)
    year_founded = Optional(int, unique=False)
    staff = Optional(lambda : Staff)
    arena = Optional(lambda : Arena)
    team_season = Set(lambda : Team_Season)

class Player(db.Entity):
    name = Required(str, 25, unique=False, nullable=False)
    age = Optional(int)
    team_season = Optional(lambda : Team_Season)
    position = Optional(str, 30, nullable=True)
    player_stats = Optional(lambda : PlayerStats, cascade_delete=True)

class Coach(db.Entity):
    name = Required(str, 25, unique=False, nullable=False)
    team_season = Required(Team_Season, unique=False)
    coach_stats = Optional(lambda : CoachStats, cascade_delete=True, unique=False)
    PrimaryKey(name, team_season)

class Staff(db.Entity):
    id = PrimaryKey(int, auto=True)
    owner = Required(str, 40, unique=False)
    manager = Optional(str, 25)
    team = Required(Team)

class TeamStats(db.Entity):
    id = PrimaryKey(int, size=8, auto=True)
    ELO = Required(float)
    FG_PCT = Required(float)
    FG3_PCT = Required(float)
    FGA = Required(float)
    FGM = Required(float)
    FG3M = Required(float)
    FG3A = Required(float)
    FG2M = Required(float)
    FG2A = Required(float)
    FG_MISSED = Required(float)
    FT_MISSED = Required(float)
    FTM = Required(float)
    FT_PCT = Required(float)
    FTA = Required(float)
    ORB = Required(float)
    DRB = Required(float)
    AST = Required(float)
    STL = Required(float)
    BLK = Required(float)
    TO = Required(float)
    PF = Required(float)
    EFFICACY = Required(float)
    PIE = Required(float)
    POSS = Required(float)
    PACE = Required(float)
    PTS = Required(float)
    team = Optional(Team_Season)


class PlayerStats(db.Entity):
    id = PrimaryKey(int, auto=True)
    GMS = Required(int)
    GStart = Required(int)
    MP = Required(int)
    FG = Required(int)
    FGA = Required(int)
    FGP = Required(float)
    ThreeP = Required(int)
    ThreePA = Required(int)
    ThreePP = Required(float)
    TwoP = Required(float)
    TwoPA = Required(float)
    TwoPP = Required(float)
    eFGP = Required(float)
    FT = Required(float)
    FTA = Required(float)
    FTP = Optional(float)
    ORB = Required(float)
    DRB = Required(float)
    TRB = Required(float)
    AST = Required(float)
    STL = Required(float)
    BLK = Required(float)
    TOV = Required(float)
    PF = Required(float)
    PTS = Required(float)
    player = Optional(Player)

class CoachStats(db.Entity):
    id = PrimaryKey(int, auto=True)
    SeasG = Required(int)
    SeasW = Required(int)
    SeasL = Optional(float)
    FranG = Required(int)
    FranW = Required(int)
    FranL = Required(int)
    CareW = Required(int)
    CareL = Required(int)
    CareWP = Required(float)
    POSeasG = Required(float)
    POSeasW = Required(float)
    POSeasL = Required(float)
    POFranW = Required(float)
    POFranG = Required(float)
    POFranL = Required(float)
    POCareG = Required(float)
    POCareW = Required(float)
    POCareL = Required(float)
    coach = Optional(Coach)

db.generate_mapping(create_tables=True)

## Let's populate the database schema just created

In [223]:
full = pd.read_csv("../CSV_Files/nba2019.csv")
playerdf = pd.read_csv("../CSV_Files/player.csv")

## Insert season 2019

In [224]:
# Def used to insert a team from processed data into database
def insert_team(data):
    return Team(id=data['Team_ID'], name=data['Name'], abbr=data['Abbr'], year_founded=data['Year_Founded'], nickname = data['Nickname'])

def insert_team_season(season, team, team_stats):
    return Team_Season(season = season, team = team, stats = team_stats)

def insert_arena(data, team):
    return Arena(name=data['Arena'], city=data['City'], capacity = data['Capacity'], team = team)

def insert_staff(data, team):
    return Staff(owner = data['Owner'], manager = data['Manager'], team = team)

# def used to insert team stats into a database
def insert_team_stats(data):
    stats = TeamStats(
        ELO = data['ELO_BEFORE'],
        FG_PCT = data['FG_PCT'],
        FG3_PCT = data['FG3_PCT'],
        FGA = data['FGA'],
        FGM = data['FGM'],
        FG3M = data['FG3M'],
        FG3A = data['FG3A'],
        FG2M = data['FG2M'],
        FG2A = data['FG2A'],
        FG_MISSED = data['FG_MISSED'],
        FT_MISSED = data['FT_MISSED'],
        FTM = data['FTM'],
        FT_PCT = data['FT_PCT'],
        FTA = data['FTA'],
        ORB = data['OREB'],
        DRB = data['DREB'],
        AST = data['AST'],
        STL = data['STL'],
        BLK = data['BLK'],
        TO = data['TO'],
        PF = data['PF'],
        EFFICACY = data['EFFICACY'],
        PIE = data['PIE'],
        POSS = data['POSS'],
        PACE = data['PACE'],
        PTS = data['PTS'],
    )
    return stats

# def used to insert a coach into a database
def insert_coach(data, tm):
    coach = Coach(
        name = data['Coach_Name'],
        team_season = tm
    )
    return coach

# def used to insert coach stats into a database e set relationship with a coach
def insert_coach_stats(data, coach: Coach):
    stats = CoachStats(
    SeasG = data['SeasG'],
    SeasW = data['SeasW'],
    SeasL = data['SeasL'],
    FranG = data['FranG'],
    FranW = data['FranW'],
    FranL = data['FranL'],
    CareW = data['CareW'],
    CareL = data['CareL'],
    CareWP = data['CareWP'],
    POSeasG = data['POSeasG'],
    POSeasW = data['POSeasW'],
    POSeasL = data['POSeasL'],
    POFranW = data['POFranW'],
    POFranG = data['POFranG'],
    POFranL = data['POFranL'],
    POCareG = data['POCareG'],
    POCareW = data['POCareW'],
    POCareL = data['POCareL'],
    coach = coach
    )
    return stats


In [225]:
# def used to insert coach stats into a database e set relationship with a coach
def insert_coach_stats_new(coach : Coach):
    stats = CoachStats(
        SeasG = coach.coach_stats.SeasG,
        SeasW = coach.coach_stats.SeasW,
        SeasL = coach.coach_stats.SeasL,
        FranG = coach.coach_stats.FranG,
        FranW = coach.coach_stats.FranW,
        FranL = coach.coach_stats.FranL,
        CareW = coach.coach_stats.CareW,
        CareL = coach.coach_stats.CareL,
        CareWP = coach.coach_stats.CareWP,
        POSeasG = coach.coach_stats.POSeasG,
        POSeasW = coach.coach_stats.POSeasW,
        POSeasL = coach.coach_stats.POSeasL,
        POFranW = coach.coach_stats.POFranW,
        POFranG = coach.coach_stats.POFranG,
        POFranL = coach.coach_stats.POFranL,
        POCareG = coach.coach_stats.POCareG,
        POCareW = coach.coach_stats.POCareW,
        POCareL = coach.coach_stats.POCareL,
    )
    return stats

In [226]:
season = Season(season = 2018, start_date = date(2018, 10, 22), end_date = date(2019, 4, 10))
season20 = Season(season = 2019, start_date = date(2019,10, 22), end_date= date(2020,10,11))
commit()

In [227]:

for index, row in full.iterrows():
    team = insert_team(row)
    arena = insert_arena(row, team)
    staff = insert_staff(row, team)
    team_stats = insert_team_stats(row)
    team.staff = staff
    team.arena = arena
    team_season = Team_Season(season=season, team=team, stats=team_stats)
    coach = insert_coach(row, team_season)
    coach_stats = insert_coach_stats(row, coach)
    coach.coach_stats = coach_stats
    commit()


In [228]:
stats20 = pd.read_csv("../CSV_Files/nba_2020.csv")

In [229]:
for index, row in stats20.iterrows():
    team = Team.select(lambda tm: tm.id == row['TEAM_ID']).get()
    ts = Team_Season.select(lambda tms : tms.team.id == row['TEAM_ID']).get()
    stats = insert_team_stats(row)
    tm_season = Team_Season(season=season20, team=team, stats=stats)
    coach = Coach(name= ts.coach.name, team_season=tm_season)
    coach_stats = insert_coach_stats_new(ts.coach)
    coach_stats.coach = coach
    coach.coach_stats = coach_stats
    commit()

In [230]:
def insert_player(data):
    player = Player(
        name = data['Name'],
        position =  data['Pos'],
        age = data['Age'],
        team_season = Team_Season.select(lambda tm : tm.team.id == data['TEAM_ID'] and tm.season.season == 2018).get()
    )
    return player

def insert_player_stats(data, player):
    stats = PlayerStats(
        GMS = data['Gms'],
        GStart = data['Gstart'],
        MP = data['MP'],
        FG = data['FG'],
        FGA = data['FGA'],
        FGP = data['FGP'],
        ThreeP = data['ThreeP'],
        ThreePA = data['ThreePA'],
        ThreePP = data['ThreePP'],
        TwoP = data['TwoP'],
        TwoPA = data['TwoPA'],
        TwoPP = data['TwoPP'],
        eFGP = data['eFGP'],
        FT = data['FT'],
        FTA = data['FTA'],
        FTP = data['FTP'],
        ORB = data['ORB'],
        DRB = data['DRB'],
        TRB = data['TRB'],
        AST = data['AST'],
        STL = data['STL'],
        BLK = data['BLK'],
        TOV = data['TOV'],
        PF = data['PF'],
        PTS = data['PTS'],
        player = player
    )
    return stats


In [231]:
for index, row in playerdf.iterrows():
    player = insert_player(row)
    stats = insert_player_stats(row, player)
    player.stats = stats
    tm = Team_Season.select(lambda t: t.team.id == row['TEAM_ID'] and t.season.season == 2018).get()
    tm.player.add(player)
    commit()

In [232]:
Team_Season.select(lambda t: t.team.id == row['TEAM_ID'] and t.season.season == 2018).show()

season      |team            |coach                               |stats        
------------+----------------+------------------------------------+-------------
Season[2018]|Team[1610612747]|Coach['Luke Walton',Team_Season[S...|TeamStats[11]


In [233]:
from pony.orm.serialization import to_dict
Player.select(lambda p: p.team_season.team.id == 1610612745 and p.team_season.season.season == 2018).show()

id |name             |age|team_season                 |position|player_stats    
---+-----------------+---+----------------------------+--------+----------------
14 |Ryan Anderson    |29 |Team_Season[Season[2018],...|PF      |PlayerStats[14] 
20 |Trevor Ariza     |32 |Team_Season[Season[2018],...|SF      |PlayerStats[20] 
53 |Tarik Black      |26 |Team_Season[Season[2018],...|C       |PlayerStats[53] 
77 |Bobby Brown      |33 |Team_Season[Season[2018],...|PG      |PlayerStats[77] 
80 |Markel Brown     |26 |Team_Season[Season[2018],...|SG      |PlayerStats[80] 
93 |Isaiah Canaan    |26 |Team_Season[Season[2018],...|PG-SG   |PlayerStats[93] 
95 |Clint Capela     |23 |Team_Season[Season[2018],...|C       |PlayerStats[95] 
199|Eric Gordon      |29 |Team_Season[Season[2018],...|SG      |PlayerStats[199]
207|Gerald Green     |32 |Team_Season[Season[2018],...|SG      |PlayerStats[207]
214|James Harden     |28 |Team_Season[Season[2018],...|SG      |PlayerStats[214]
241|Nene Hilario     |35 |Te

## CRUD Operation on data

In [296]:
@db_session
def get_all_team_of_season(season:int):
    return Team_Season.select(lambda tm : tm.season.season == season)

@db_session
def get_all_season():
    return Season.select()

@db_session
def get_team(idx:int):
    return Team.select(lambda t: t.id == idx).get()

@db_session
def get_team_stats(idx:int, season:int):
    return TeamStats.select(lambda ts: ts.team.team.id == idx and ts.team.season.season == season).get()

@db_session
def get_coach(idx:int, season:int):
    return Coach.select(lambda c: c.team_season.team.id == idx and c.team_season.season.season == season ).get()

@db_session
def get_coach_stats(idx:str):
    return CoachStats.select(lambda c: c.coach.name == idx).get()

@db_session
def get_players(idx:int, season:int):
    return Player.select(lambda p: p.team_season.team.id == idx and p.team_season.season.season == season)

@db_session
def get_player_stats(idx:int):
    return PlayerStats.select(lambda ps: ps.player.id == idx).get()

@db_session
def get_team_staff(idx:int):
    return Staff.select(lambda s : s.team.id == idx).get()

@db_session
def get_arena(idx:int):
    return Arena.select(lambda a : a.team.id == idx).get()

def get_max_capacity_arena():
    return Arena.select().order_by(desc(Arena.capacity))[:5]

In [297]:
## Implementation of FastAPI to create API of nba data

In [301]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from loguru import logger
from pony.orm.serialization import to_dict

app = FastAPI()

@app.get("/team/{teamID}/arena")
@db_session
def g_arena(teamID:int):
    arena = get_arena(teamID)
    return {
        'city' : arena.city,
        'name': arena.name,
        'capacity': arena.capacity
    }

@app.get("/team/{teamID}/staff")
@db_session
def g_staff(teamID:int):
    staff = get_team_staff(teamID)
    return {
        'owner' : staff.owner,
        'manager': staff.manager,
    }

@app.get('/team/{idx}')
@db_session
def g_team(idx:int, request:Request):
    el = get_team(idx)
    data = {
        'id' : el.id,
        'name': el.name,
        'abbr': el.abbr,
        'nickname' : el.nickname,
        'year_founded' : el.year_founded,
        'arena' : request.url_for("g_arena", teamID=el.id),
        'staff': request.url_for("g_staff", teamID=el.id)
    }

    return data

@app.get("/{coach}/stats")
@db_session
def g_coach_stats(coach:str):
    c = get_coach_stats(coach)
    data = {}
    for key,value in c.to_dict().items():
        if key == "coach" or key == "id":
            continue
        data[key] = value
    return data

@app.get("/{season}/team/{teamID}/coach")
@db_session
def g_coach(teamID:int, season:int, request:Request):
    c = get_coach(teamID, season)
    return {
        "name" : c.name,
        "stats" : request.url_for('g_coach_stats', coach=c.name)
    }

@app.get("/{season}/{teamID}/stats")
@db_session
def g_team_stats(season:int, teamID:int):
    c = get_team_stats(teamID, season)
    data = {}
    for key,value in c.to_dict().items():
        if key == "team" or key == "id":
            continue
        data[key] = value
    return data

@app.get("/{season}/{teamID}/{playerID}/stats")
@db_session
def g_player_stats(season:int, teamID:int, playerID:int):
    ps = get_player_stats(playerID)
    data = {}
    for key,value in ps.to_dict().items():
        if key == "player":
            continue
        data[key] = value
    return data

@app.get("/{season}/{teamID}/players")
@db_session
def g_players(season:int, teamID:int, request:Request):
    p = get_players(teamID, season)
    data = []
    for el in p:
        data.append({
            'name' : el.name,
            'age' : el.age,
            'position' : el.position,
            'stats' : request.url_for('g_player_stats', season=season, teamID=teamID, playerID=el.id)
        })
    return data

@app.get("/{season}")
@db_session
def all_season(request: Request, season:int):
    season_team = get_all_team_of_season(2018)
    data = []
    for el in season_team:
        data.append({
            'Team' : el.team.name,
            'team_info' : request.url_for("g_team", idx=el.team.id),
            'stats' : request.url_for('g_team_stats', teamID=el.team.id, season=el.season.season),
            'coach' : request.url_for('g_coach', teamID=el.team.id, season=el.season.season),
            'players' : request.url_for('g_players', teamID=el.team.id, season=el.season.season)

        })
    return data

In [302]:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(3000)

ngrok_tunnel

<NgrokTunnel: "http://2027-185-63-103-73.ngrok.io" -> "http://localhost:3000">

In [303]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=3000)

INFO:     Started server process [94722]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62598 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62598 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:62599 - "GET /2018 HTTP/1.1" 200 OK
INFO:     127.0.0.1:62621 - "GET /2018 HTTP/1.1" 200 OK
INFO:     127.0.0.1:62629 - "GET /team/1610612744 HTTP/1.1" 200 OK
INFO:     127.0.0.1:62642 - "GET /team/1610612744/arena HTTP/1.1" 200 OK
INFO:     127.0.0.1:62678 - "GET /2018/team/1610612744/coach HTTP/1.1" 200 OK
INFO:     127.0.0.1:62705 - "GET /2018 HTTP/1.1" 200 OK
INFO:     127.0.0.1:62711 - "GET /2018/1610612744/players HTTP/1.1" 200 OK
INFO:     127.0.0.1:62736 - "GET /2018/1610612744/133/stats HTTP/1.1" 200 OK
INFO:     127.0.0.1:62750 - "GET /2018/1610612744/stats HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [94722]
